In [ ]:
import json
import sqlite3
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

DATABASE_NAME = 'database.db'
TABLE_NAME = 'BAKERIES'

engine = create_engine(f'sqlite:///{DATABASE_NAME}', echo=False)

# Initialize the database
def init_db():
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            FirstName TEXT NOT NULL,
            LastName TEXT NOT NULL,
            NID TEXT NOT NULL,
            City TEXT NOT NULL,
            Region TEXT NOT NULL,
            District TEXT NOT NULL,
            Lat REAL NOT NULL,
            Lon REAL NOT NULL,
            HouseholdRisk TEXT NOT NULL,
            BakersRisk TEXT NOT NULL,
            TypeFlour INTEGER NOT NULL,
            TypeBread TEXT NOT NULL,
            BreadRations INTEGER NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

init_db()

df = pd.read_csv("./assets/data/data.csv")
gdf_regions = gpd.read_file('./assets/data/geodatabase/Region.geojson')
gdf_district = gpd.read_file('./assets/data/geodatabase/District.geojson')

# FirstName, LastName, NID, City, Lat, Lon, HouseholdRisk, BakersRisk, TypeFlour, TypeBread, BreadRations
COLs = ['FirstName', 'LastName', 'City', 'HouseholdRisk', 'BakersRisk', 'TypeBread']
df[COLs] = df[COLs].astype(str)
df[COLs] = df[COLs].apply(lambda x: x.str.rstrip())
df[COLs] = df[COLs].apply(lambda x: x.str.lstrip())
df[COLs] = df[COLs].apply(lambda x: x.str.replace(' +', ' '))
df[COLs] = df[COLs].apply(lambda x: x.str.replace('ي','ی'))
df[COLs] = df[COLs].apply(lambda x: x.str.replace('ئ','ی'))
df[COLs] = df[COLs].apply(lambda x: x.str.replace('ك', 'ک'))

COLs = ['NID', 'TypeFlour', 'BreadRations']
df[COLs] = df[COLs].astype(int)

COLs = ['Lat', 'Lon']
df[COLs] = df[COLs].astype(float)

# Drop All NULL Value from Lat & Lon Columns
df.dropna(subset=['Lat', 'Lon'], inplace=True)

# # Remove Duplicate NID
# df.drop_duplicates(subset=['NID'], inplace=True)

# # Remove Same Lat & Lon
# df.drop_duplicates(subset=['Lat', 'Lon'], inplace=True)

# Convert NID to `str`
df['NID'] = df['NID'].apply(lambda x: str(x).zfill(10))

# Reset Index
df.reset_index(drop=True, inplace=True)

gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['Lon'], df['Lat'])
)

gdf = gdf.set_crs('EPSG:4326')
gdf_regions = gdf_regions.to_crs('EPSG:4326')
gdf_district = gdf_district.to_crs('EPSG:4326')

gdf_joined_gdf_regions = gpd.sjoin(gdf, gdf_regions, how='left', op='within')
gdf_joined_gdf_district = gpd.sjoin(gdf, gdf_district, how='left', op='within')

df['Region'] = gdf_joined_gdf_regions['Region']
df['District'] = gdf_joined_gdf_district['District']

# Drop All NULL Value from Region & Lon District
df.dropna(subset=['Region', 'District'], inplace=True)

df.drop_duplicates(inplace=True)

# Save to Database
df.to_sql(name='BAKERIES', con=engine, if_exists='replace', index=False)

In [46]:
import json
import sqlite3
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

df = pd.read_csv("./data.csv")
gdf_regions = gpd.read_file('./app/assets/data/geodatabase/Region.geojson')
gdf_district = gpd.read_file('./app/assets/data/geodatabase/District.geojson')

COLs = ['first_name', 'last_name', 'ownership_status', 'second_fuel', 'city', 'household_risk', 'bakers_risk', 'type_bread', 'nid', 'phone', 'bakery_id']
df[COLs] = df[COLs].astype(str)
df[COLs] = df[COLs].apply(lambda x: x.str.rstrip())
df[COLs] = df[COLs].apply(lambda x: x.str.lstrip())
df[COLs] = df[COLs].apply(lambda x: x.str.replace(' +', ' '))
df[COLs] = df[COLs].apply(lambda x: x.str.replace('ي','ی'))
df[COLs] = df[COLs].apply(lambda x: x.str.replace('ئ','ی'))
df[COLs] = df[COLs].apply(lambda x: x.str.replace('ك', 'ک'))

COLs = ['number_violations', 'type_flour', 'bread_rations']
df[COLs] = df[COLs].astype(int, errors='ignore')

COLs = ['lat', 'lon']
df[COLs] = df[COLs].astype(float)


# Drop All NULL Value from Lat & Lon Columns
df.dropna(subset=['lat', 'lon'], inplace=True)

# Remove All Duplicates Row
df.drop_duplicates(inplace=True)

# Convert nid and phone to `str`
df['nid'] = df['nid'].apply(lambda x: str(x).zfill(10))
df['phone'] = df['phone'].apply(lambda x: str(x).zfill(11))

# # Remove Duplicate NID
# df.drop_duplicates(subset=['NID'], inplace=True)

# # Remove Same Lat & Lon
# df.drop_duplicates(subset=['Lat', 'Lon'], inplace=True)

# Reset Index
df.reset_index(drop=True, inplace=True)

gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['lon'], df['lat'])
)

gdf = gdf.set_crs('EPSG:4326')
gdf_regions = gdf_regions.to_crs('EPSG:4326')
gdf_district = gdf_district.to_crs('EPSG:4326')

COLs = ['first_name', 'last_name', 'nid', 'phone', 'bakery_id',
       'ownership_status', 'number_violations', 'second_fuel', 'city', 'lat',
       'lon', 'household_risk', 'bakers_risk', 'type_flour', 'type_bread',
       'bread_rations']
gdf_joined_gdf_regions = gpd.sjoin(gdf, gdf_regions, how='left', op='within')
gdf_joined_gdf_regions.drop_duplicates(subset=COLs, inplace=True)
gdf_joined_gdf_district = gpd.sjoin(gdf, gdf_district, how='left', op='within')
gdf_joined_gdf_district.drop_duplicates(subset=COLs, inplace=True)

df['region'] = gdf_joined_gdf_district['region']
df['district'] = gdf_joined_gdf_district['district']

# Drop All NULL Value from Region & Lon District
df.dropna(subset=['region', 'district'], inplace=True)

COLs = ['first_name', 'last_name', 'ownership_status', 'second_fuel', 'city', 'household_risk', 'bakers_risk', 'type_bread', 'nid', 'phone', 'bakery_id']
df[COLs] = df[COLs].astype(str)

COLs = ['number_violations', 'type_flour', 'bread_rations', 'region', 'district']
df[COLs] = df[COLs].astype(int, errors='ignore')

COLs = ['lat', 'lon']
df[COLs] = df[COLs].astype(float)

# Reset Index
df.reset_index(drop=True, inplace=True)

df.info()
# Save to Database
# df.to_sql(name='BAKERIES', con=engine, if_exists='replace', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1765 entries, 0 to 1764
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   first_name         1765 non-null   object 
 1   last_name          1765 non-null   object 
 2   nid                1765 non-null   object 
 3   phone              1765 non-null   object 
 4   bakery_id          1765 non-null   object 
 5   ownership_status   1765 non-null   object 
 6   number_violations  1765 non-null   int32  
 7   second_fuel        1765 non-null   object 
 8   city               1765 non-null   object 
 9   lat                1765 non-null   float64
 10  lon                1765 non-null   float64
 11  household_risk     1765 non-null   object 
 12  bakers_risk        1765 non-null   object 
 13  type_flour         1765 non-null   int32  
 14  type_bread         1765 non-null   object 
 15  bread_rations      1765 non-null   int32  
 16  region             1765 

c:\Users\Pooya\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
c:\Users\Pooya\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [45]:
df

,first_name,last_name,nid,phone,bakery_id,ownership_status,number_violations,second_fuel,city,lat,lon,household_risk,bakers_risk,type_flour,type_bread,bread_rations,region,district
1,مجتبی,پیری دیسفانی,83093079.0,9151001000.0,100100.0,مالک,2,گازوییل,مشهد,36.396108,59.673987,ریسک متوسط,ریسک متوسط,3,لواش,225,3,4
2,علیرضا,جمالی بجستانی,932034810.0,9151001000.0,100100.0,استیجاری,3,نفت,مشهد,36.278046,59.616124,کم ریسک,کم ریسک,4,تافتون,423,7,1
4,مهدی,نیستانی,670487198.0,9151001000.0,100100.0,استیجاری,3,نفت,مشهد,36.311592,59.593240,ریسک متوسط,کم ریسک,6,تافتون,173,2,1
5,طیبه,حسینی,5228933786.0,9151001000.0,100100.0,مالک,4,ندارد,مشهد,36.273543,59.598718,کم ریسک,خیلی پرریسک,6,بربری,166,8,3
6,حسن,موسی پورقره تکان,5539909356.0,9151001000.0,100100.0,استیجاری,5,گازوییل,مشهد,36.337001,59.537757,خیلی پرریسک,پرریسک,6,سنگک,174,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2283,محمدرضا,جعفری سرنابادی,748889991.0,9151001000.0,100100.0,استیجاری,5,گازوییل,مشهد,36.321290,59.620926,پرریسک,پرریسک,1,سنگک,118,3,1
2284,حمید,آصفی جزین,5620001032.0,9151001000.0,100100.0,مالک,0,نفت,مشهد,36.271869,59.672788,ریسک متوسط,ریسک متوسط,1,لواش,125,6,3
2285,اسماعیل,ملا حیدری سرچشمه,920914586.0,9151001000.0,100100.0,استیجاری,1,ندارد,مشهد,36.320631,59.485033,کم ریسک,کم ریسک,2,تافتون,122,9,3
2286,حسن,هوشیارنقاب,5228915907.0,9151001000.0,100100.0,مالک,2,گازوییل,مشهد,36.290558,59.524135,خیلی پرریسک,خیلی پرریسک,3,بربری,118,9,1


In [40]:
gdf_joined_gdf_district.columns

Index(['first_name', 'last_name', 'nid', 'phone', 'bakery_id',
       'ownership_status', 'number_violations', 'second_fuel', 'city', 'lat',
       'lon', 'household_risk', 'bakers_risk', 'type_flour', 'type_bread',
       'bread_rations', 'geometry', 'index_right', 'fid', 'region',
       'district'],
      dtype='object')

In [35]:
gdf_joined_gdf_regions

,first_name,last_name,nid,phone,bakery_id,ownership_status,number_violations,second_fuel,city,lat,lon,household_risk,bakers_risk,type_flour,type_bread,bread_rations,geometry,index_right,fid,region
0,نرگس,رحیمی,930807847.0,9151001000.0,100100.0,مالک,0.0,نفت,مشهد,25.250000,35.350000,خیلی پرریسک,خیلی پرریسک,1.0,بربری,69.0,POINT (35.35000 25.25000),NaN,NaN,NaN
1,مجتبی,پیری دیسفانی,83093079.0,9151001000.0,100100.0,مالک,2.0,گازوییل,مشهد,36.396108,59.673987,ریسک متوسط,ریسک متوسط,3.0,لواش,225.0,POINT (59.67399 36.39611),3.0,4.0,3.0
2,علیرضا,جمالی بجستانی,932034810.0,9151001000.0,100100.0,استیجاری,3.0,نفت,مشهد,36.278046,59.616124,کم ریسک,کم ریسک,4.0,تافتون,423.0,POINT (59.61612 36.27805),8.0,9.0,7.0
3,بهروز,کاملان کافی,943234670.0,9151001000.0,100100.0,مالک,4.0,ندارد,مشهد,35.681018,51.506991,خیلی پرریسک,خیلی پرریسک,5.0,بربری,366.0,POINT (51.50699 35.68102),NaN,NaN,NaN
4,مهدی,نیستانی,670487198.0,9151001000.0,100100.0,استیجاری,3.0,نفت,مشهد,36.311592,59.593240,ریسک متوسط,کم ریسک,6.0,تافتون,173.0,POINT (59.59324 36.31159),1.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2283,محمدرضا,جعفری سرنابادی,748889991.0,9151001000.0,100100.0,استیجاری,5.0,گازوییل,مشهد,36.321290,59.620926,پرریسک,پرریسک,1.0,سنگک,118.0,POINT (59.62093 36.32129),2.0,3.0,3.0
2284,حمید,آصفی جزین,5620001032.0,9151001000.0,100100.0,مالک,0.0,نفت,مشهد,36.271869,59.672788,ریسک متوسط,ریسک متوسط,1.0,لواش,125.0,POINT (59.67279 36.27187),7.0,8.0,6.0
2285,اسماعیل,ملا حیدری سرچشمه,920914586.0,9151001000.0,100100.0,استیجاری,1.0,ندارد,مشهد,36.320631,59.485033,کم ریسک,کم ریسک,2.0,تافتون,122.0,POINT (59.48503 36.32063),10.0,11.0,9.0
2286,حسن,هوشیارنقاب,5228915907.0,9151001000.0,100100.0,مالک,2.0,گازوییل,مشهد,36.290558,59.524135,خیلی پرریسک,خیلی پرریسک,3.0,بربری,118.0,POINT (59.52414 36.29056),10.0,11.0,9.0


In [ ]:
# tmp = df[[
#     'City', 'Region', 'District',
#     'HouseholdRisk', 'BakersRisk', 'TypeFlour', 'TypeBread', 'BreadRations'
# ]]

tmp = df[[
    'City', 'Region', 'District', 'TypeBread'
]]

def build_hierarchy(data, keys):
    if not keys:
        return {'size': len(data)}
    result = []
    for key, group in data.groupby(keys[0]):
        
        if key in list(data['TypeBread'].unique()):
            result.append({
                'name': key,
                'size': len(group['TypeBread'])
            })
        else:
            result.append({
                'name': key,
                'children': build_hierarchy(group, keys[1:]) if len(keys) > 1 else len(group['TypeBread'])
            })
            
    return result

# Build the hierarchical structure
# hierarchy = build_hierarchy(tmp, ['City', 'Region', 'District', 'HouseholdRisk', 'BakersRisk', 'TypeFlour', 'TypeBread', 'BreadRations'])
hierarchy = build_hierarchy(tmp, ['City', 'Region', 'District', 'TypeBread'])

# Convert to JSON
hierarchical_json = {'name': 'Root', 'children': hierarchy}

hierarchical_json

In [11]:
with open('sunburst_data.json', 'w') as json_file:
    json.dump(hierarchical_json, json_file, indent=4)